In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '1'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'trentino'
sub_set_name = 'internet'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.070513,0.079877,0.138498,0.079829,0.086237,0.081288,0.088334,0.153213,0.122837,0.126910,...,0.038866,0.015604,0.019802,0.023932,0.060878,0.019350,0.020722,0.070202,0.055282,0.019964
2013-11-01 01:00:00,0.049713,0.060607,0.092762,0.048852,0.053276,0.066329,0.069117,0.109108,0.088937,0.093735,...,0.024250,0.012212,0.021476,0.014672,0.035150,0.024047,0.025600,0.038681,0.033044,0.028545
2013-11-01 02:00:00,0.059755,0.064257,0.056337,0.050885,0.064577,0.073372,0.063995,0.088428,0.075936,0.092412,...,0.017183,0.008558,0.013205,0.013320,0.023523,0.013948,0.013838,0.027604,0.022939,0.014725
2013-11-01 03:00:00,0.052892,0.053441,0.038942,0.046973,0.047012,0.056657,0.049539,0.064613,0.054763,0.058460,...,0.013630,0.004955,0.006178,0.009959,0.026692,0.007357,0.011778,0.031836,0.020744,0.016335
2013-11-01 04:00:00,0.040440,0.055874,0.018405,0.048165,0.053330,0.060869,0.050811,0.061678,0.040994,0.041741,...,0.010115,0.005666,0.008015,0.010488,0.026558,0.008772,0.010915,0.032037,0.019349,0.015189


In [7]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.070513,0.079877,0.138498,0.079829,0.086237,0.081288,0.088334,0.153213,0.122837,0.126910,...,0.038866,0.015604,0.019802,0.023932,0.060878,0.019350,0.020722,0.070202,0.055282,0.019964
2013-11-01 01:00:00,0.049713,0.060607,0.092762,0.048852,0.053276,0.066329,0.069117,0.109108,0.088937,0.093735,...,0.024250,0.012212,0.021476,0.014672,0.035150,0.024047,0.025600,0.038681,0.033044,0.028545
2013-11-01 02:00:00,0.059755,0.064257,0.056337,0.050885,0.064577,0.073372,0.063995,0.088428,0.075936,0.092412,...,0.017183,0.008558,0.013205,0.013320,0.023523,0.013948,0.013838,0.027604,0.022939,0.014725
2013-11-01 03:00:00,0.052892,0.053441,0.038942,0.046973,0.047012,0.056657,0.049539,0.064613,0.054763,0.058460,...,0.013630,0.004955,0.006178,0.009959,0.026692,0.007357,0.011778,0.031836,0.020744,0.016335
2013-11-01 04:00:00,0.040440,0.055874,0.018405,0.048165,0.053330,0.060869,0.050811,0.061678,0.040994,0.041741,...,0.010115,0.005666,0.008015,0.010488,0.026558,0.008772,0.010915,0.032037,0.019349,0.015189


In [8]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path='./dataset/trentino/'
args.data_path='internet.csv'
args.data_path

'internet.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data[0][0].shape

(96, 6259)

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    print(train_data[0][0].shape)

(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 625)
(96, 634)


In [13]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
        

# Setup WANDB

In [14]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Trentino Internet dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 3.9608805179595947
Epoch: 2 cost time: 2.6356217861175537
Epoch: 3 cost time: 2.5792269706726074
Epoch: 4 cost time: 2.6608664989471436
Epoch: 5 cost time: 2.6010007858276367
Epoch: 1 cost time: 2.545443058013916
Epoch: 2 cost time: 2.5215184688568115
Epoch: 3 cost time: 2.4727935791015625
Epoch: 4 cost time: 2.924386739730835
Epoch: 5 cost time: 3.210320472717285


Progress:   0%|          | 0/50 [00:42<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1268 and 1250x64)

wandb: Network error (ReadTimeout), entering retry loop.
